In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, Activation, Flatten, Dropout, Conv2D, MaxPooling2D
import matplotlib.pyplot as plt

# Dataset

In [3]:
expert_1_path = r'C:\Users\HP\Desktop\New folder (2)\MedicalExpert-I'
expert_2_path = r'C:\Users\HP\Desktop\New folder (2)\MedicalExpert-II'

In [4]:
img_size = 256
categories = ["Normal", "Doubtful", "Mild", "Moderate", "Severe"]

# Data Loading

In [6]:
def load_data_from_expert(expert_path):
    data = []
    labels = []
    label_dict = {category: idx for idx, category in enumerate(categories)}

    for category in categories:
        folder_path = os.path.join(expert_path, category)
        img_names = os.listdir(folder_path)
        
        for img_name in img_names:
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path)
            try:
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                resized = cv2.resize(gray, (img_size, img_size))
                data.append(resized)
                labels.append(label_dict[category])
            except Exception as e:
                print(f'Exception: {e} in {img_path}')
    
    return np.array(data), np.array(labels)

In [7]:
x_data_1, y_data_1 = load_data_from_expert(expert_1_path)
x_data_2, y_data_2 = load_data_from_expert(expert_2_path)

# Data Processing

In [9]:
x_data = np.concatenate((x_data_1, x_data_2), axis=0)
y_data = np.concatenate((y_data_1, y_data_2), axis=0)

In [10]:
x_data = x_data / 255.0
x_data = np.reshape(x_data, (x_data.shape[0], img_size, img_size, 1))

In [11]:
y_data = to_categorical(y_data, num_classes=5)

# Creating the model

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

In [14]:
from keras.models import Sequential
model = Sequential()
model.add(Conv2D(128, (3, 3), input_shape=(img_size, img_size, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dense(5, activation='softmax'))  

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

C:\Users\HP\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 254, 254, 128)       │           1,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 254, 254, 128)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 127, 127, 128)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 125, 125, 64)        │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 125, 125, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 60, 60, 32)          │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 60, 60, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 30, 30, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 28800)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 28800)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,686,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 5)                   │             325 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,788,645 (14.45 MB)

 Trainable params: 3,788,645 (14.45 MB)

 Non-trainable params: 0 (0.00 B)

# Training

In [ ]:
a = model.fit(x_data, y_data, epochs=25, validation_split=0.2)

Epoch 1/25
83/83 ━━━━━━━━━━━━━━━━━━━━ 648s 8s/step - accuracy: 0.3695 - loss: 1.4042 - val_accuracy: 0.0015 - val_loss: 2.6453
Epoch 2/25
83/83 ━━━━━━━━━━━━━━━━━━━━ 597s 7s/step - accuracy: 0.4175 - loss: 1.3258 - val_accuracy: 0.0273 - val_loss: 2.2543
Epoch 3/25
83/83 ━━━━━━━━━━━━━━━━━━━━ 617s 7s/step - accuracy: 0.4610 - loss: 1.2612 - val_accuracy: 0.0818 - val_loss: 2.0204
Epoch 4/25
83/83 ━━━━━━━━━━━━━━━━━━━━ 593s 7s/step - accuracy: 0.4759 - loss: 1.2531 - val_accuracy: 0.0364 - val_loss: 2.0917
Epoch 5/25
83/83 ━━━━━━━━━━━━━━━━━━━━ 639s 8s/step - accuracy: 0.5012 - loss: 1.1933 - val_accuracy: 0.1136 - val_loss: 1.8279
Epoch 6/25
83/83 ━━━━━━━━━━━━━━━━━━━━ 592s 7s/step - accuracy: 0.4997 - loss: 1.1786 - val_accuracy: 0.1000 - val_loss: 1.9793
Epoch 7/25
83/83 ━━━━━━━━━━━━━━━━━━━━ 587s 7s/step - accuracy: 0.5216 - loss: 1.1415 - val_accuracy: 0.2288 - val_loss: 1.9375
Epoch 8/25
83/83 ━━━━━━━━━━━━━━━━━━━━ 587s 7s/step - accuracy: 0.5697 - loss: 1.0551 - val_accuracy: 0.2803 - v

# Training Loss and Accuracy 
# Testing Loss and Accuracy

In [ ]:
train_loss, train_accuracy = model.evaluate(x_train, y_train)
print(f"Train Loss: {train_loss}")
print(f"Train Accuracy: {train_accuracy}")
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
metrics = ['Train Loss', 'Test Loss', 'Train Accuracy', 'Test Accuracy']
values = [train_loss, test_loss, train_accuracy, test_accuracy]
plt.figure(figsize=(6, 6))
bars = plt.bar(metrics, values, color=['blue', 'orange', 'green', 'red'])
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval, round(yval, 4), ha='center', va='bottom')

plt.title('Final Train/Test Loss and Accuracy')
plt.ylabel('Value')
plt.show()

# Confusion Matrix and Classification Report

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from mlxtend.plotting import plot_confusion_matrix
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)
cm = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(10, 8))
plot_confusion_matrix(cm, figsize=(10, 8), cmap=plt.cm.Blues, hide_ticks=True)
plt.xticks(range(5), categories, fontsize=12)
plt.yticks(range(5), categories, fontsize=12)
plt.title('Confusion Matrix', fontsize=16)
plt.show()
class_report = classification_report(y_true_classes, y_pred_classes, target_names=categories)
print("Classification Report:\n", class_report)

In [ ]:
X = 32
img_size = 256
img_single = x_test[X]
img_single = cv2.resize(img_single, (img_size, img_size))
img_single = np.expand_dims(img_single, axis=0)
img_single = img_single.reshape(img_single.shape[0], img_size, img_size, 1)
predictions_single = model.predict(img_single)
print('Predicted Image is :', categories[np.argmax(predictions_single)])
print("Correct label is:", categories[np.argmax(y_test[X])])
plt.imshow(np.squeeze(img_single), cmap='gray')
plt.grid(False)
plt.show()